# Aim

Some notes about using Bokeh for visualisation. Note that there is an R compatibility layer.

Environment: Python3.6 (bokeh-py3)

# Notes

## Concepts

Bokeh is based around a set of idioms: 

* A **plot** (or confusingly sometimes *figures*) is what you draw on. It has axes, labels, legends, scales and so on. Think of it as everything a plot has that isn't the data.
* **Glyphs** are the drawing primitives: lines, circles, etc. Again, confusingly, Bokeh also talks of *markers*.
* A **renderer** is the single drawing call, that may actually draw a number of things
* Plots can be decorated with **guides** (visual depections of distance like grids and axes) and **annotations** (labels, legends, etc.)
* **Ranges** are what you expect. Bokeh autogenerates these but they can also be converted from obvious pairs like `(0, 10)` or `[-0.5, 0.75]`.
* A plot is thus a collection (graph) of objects that may be named, selected (by type or name) and manipulated.
* Colours can be expressed as:
  * named CSS colors, e.g 'green', 'indigo'
  * RGB(A) hex value, e.g., '#FF0000', '#44444444'
  * 3-tuples of integers (r,g,b) between 0 and 255
  * 4-tuple of (r,g,b,a) where r, g, b are integers between 0 and 255 and a is a floating point value between 0 and 1


## Basic use

A simple example, taken from the Bokeh quickstart. This produces a nice figure (zoomable, pannable, saveable) inline:

In [19]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

# the data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# output inline to notebook
# You can also use Apache Zepplin notebooks with output_notebook(notebook_type=’zeppelin’)
output_notebook()

# create a new plot with title and axis labels
fig = figure (title="simple line example", x_axis_label='x', y_axis_label='y')

# add a line to plot
# - note how line thickness is set
# - names to be used in legend given
fig.line (x, y, legend="Temp.", line_width=2)

# show the results
show (fig)

Loading BokehJS ...

Got some weirdness here where the previous save-to-static-file call is still being used. It stopped after a while so ...

It seems the `output_X` call can be called almost any time before the `show` call.

The `figure` call sets up the area to be drawn in (the plot), with ranges, labels and scales.

You then call `line` (or other "renderers") to actually draw things (the data) on the figure.

A more complex example:

In [20]:
# prepare some data
x = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
y0 = [i**2 for i in x]
y1 = [10**i for i in x]
y2 = [10**(i**2) for i in x]

# once output_foo has been called, everything is sent to that destination

# create a new plot
# so you can choose the tools available 
p = figure (
   tools="resize,crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select",
   y_axis_type="log", y_range=[0.001, 10**11], title="log axis example",
   x_axis_label='sections', y_axis_label='particles',
   responsive=True, # fill the container
)

# Add various renderers
# - note how the legend represents the lines correctly
# - interesting that to get a line-with-circles plot, you create the two
#   two seperately and gives them the same data and title
p.line(x, x, legend="y=x")
p.circle(x, x, legend="y=x", fill_color="white", size=8)
p.line(x, y0, legend="y=x^2", line_width=3)
p.line(x, y1, legend="y=10^x", line_color="red")
p.circle(x, y1, legend="y=10^x", fill_color="red", line_color="red", size=6)
p.line(x, y2, legend="y=10^x^2", line_color="orange", line_dash="4 4")


# show the results
show(p)

## Styling

In [24]:
# you can give specific plot dimensions. although bokeh generally makes better choices
p = figure (plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

# change some things about the x-axes
p.xaxis.axis_label = "Temp"
p.xaxis.axis_line_width = 3
p.xaxis.axis_line_color = (200, 100, 100)

# change just some things about the y-axes
p.yaxis.axis_label = "Pressure"
p.yaxis.major_label_text_color = "#aa6666"
p.yaxis.major_label_orientation = "vertical"

# note how you can change things on all axes
p.axis.minor_tick_in = -3
p.axis.minor_tick_out = 6

# do some title styling
p.title.text_font_size = "3.0em" # can be px, em or pt
p.title.align = 'center'
p.title.text_color = "olive"
p.title.text_font = "times, georgia"
p.title.text_font_style = "italic"

# set the background of the plot
p.background_fill_color = "beige"
p.background_fill_alpha = 0.5

# set the border (region outside the plot)
p.border_fill_color = "whitesmoke"
p.min_border_left = p.min_border_right = p.min_border_top = p.min_border_bottom = 80

show(p)

## Tools

Plot tools come in three basic types, determined by interaction. Only one one any type can be active at any point (and is highlighted in the toolbar):

  * Pan / drag: move around the plot or select parts of plot
    * `box_select`: select (highlight) points with a box
    * `lasso_select`: select points in an irregular area
    * `box_zoom`: zoom in on a section of data
    * `pan`, `xpan`, `ypan`: pan side to side and up and down in data
    * resize: resize the whole plot
  * Click / tap: using mouse clicks
    * `tap`: select individual points
    * `poly_select`: select points in an arbitrary polygon
  * Scroll / pinch: use the scrollwheel on a mouse
    * 'wheel_zoom', 'xwheel_zoom', 'ywheel_zoom': zoom in and out
    * 'xwheel_pan', 'ywheel_pan': pan vertically or horizontally
  * Actions: happen instantly when button clicked
    * undo
    * redo
    * reset: reet to original / normal viz
    * 'zoom_in', 'xzoom_in', 'yzoom_in'
    * 'zoom_out', 'xzoom_out', 'yzoom_out'
    * crosshair: a plot resizer that works by zooming in on a point?
    * save


In [26]:
# use the above data
x = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
y0 = [i**2 for i in x]
y1 = [10**i for i in x]
y2 = [10**(i**2) for i in x]

# create a new plot

p = figure (
    # choose tools available
    # note this doesn't choose the order
    tools="resize,crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select",
    # choose toolbar location
    toolbar_location="above",
    title="tool example",
)

# Add various renderers
p.line (x, x, legend="y=x")
p.circle (x, x, legend="y=x", fill_color="white", size=8)
p.line (x, y0, legend="y=x^2", line_width=3)
p.line (x, y1, legend="y=10^x", line_color="red")
p.circle (x, y1, legend="y=10^x", fill_color="red", line_color="red", size=6)
p.line (x, y2, legend="y=10^x^2", line_color="orange", line_dash="4 4")


# show the results
show(p)

## Export & saving

Headless export is possible but requires other dependencies to be installed (see http://bokeh.pydata.org/en/latest/docs/user_guide/export.html).

Alternatively, PhantomJS (via a Chrome bookmarklet) can be used to take a PDf screenshot.

In [28]:
# output to a static HTML file
output_file ("plot.html")

# everything elese is done as per normal
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

file_fig = figure()
file_fig.line (x, y)

# create & open the external file
show (file_fig)

# Random notes

* There's also Bokeh for Scala & Julia
* HTML will load the JS from a CDN. However there are ways of packaging a self-contained page
* There are various attributes you can set on a plot to improve performance, mostly needed if you're interactiing wiht a plot containing many points.